In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np

In [2]:
# baca file train dan test
train_lines = []
test_lines = []
with open('kalimat_POS_NE_train.txt', 'r') as f:
    train_lines = f.readlines()
with open('kalimat_POS_NE_test.txt', 'r') as f:
    test_lines = f.readlines()

In [3]:
# fungsi untuk konversi label NE
def convert_label(raw_ne):
    new_label = 0
    if (raw_ne=='B-PER'): new_label = 1
    if (raw_ne=='I-PER'): new_label = 2
    if (raw_ne=='B-ORG'): new_label = 3
    if (raw_ne=='I-ORG'): new_label = 4
    if (raw_ne=='B-LOC'): new_label = 5
    if (raw_ne=='I-LOC'): new_label = 6
    return new_label

In [4]:
# inisialisasi kode token dan postag
token_dict = {}
postag_dict = {}

#added prefix dict
prefix1_dict = {}
prefix2_dict = {}
prefix3_dict = {}

#added suffix dict
suffix1_dict = {}
suffix2_dict = {}
suffix3_dict = {}

#added postag dict
postag_dict = {}

counter_token = 0
counter_postag = 0

#added counter
counter_prefix1 = 0
counter_prefix2 = 0
counter_prefix3 = 0
counter_suffix1 = 0
counter_suffix2 = 0
counter_suffix3 = 0

In [5]:
# baca data token, postag, dan label NE
train_sents = []
test_sents = []
sent = []
counter = 0

In [6]:
# data train
for line in train_lines:
    line = line.rstrip('\n')
    curr_tuple = ()
    if len(line)>1:
        line_part = line.split(" ")
        t = (line_part[0], line_part[1], convert_label(line_part[2]))
        if line_part[0].lower() not in token_dict.keys():
            token_dict[line_part[0].lower()] = counter_token
            counter_token = counter_token+1
            
            if len(line_part[0].lower())>=3:
                #prefix
                if line_part[0].lower()[0] not in prefix1_dict.keys():
                    prefix1_dict[line_part[0].lower()[0]] = counter_prefix1
                    counter_prefix1 = counter_prefix1+1
                if line_part[0].lower()[0:2] not in prefix2_dict.keys():
                    prefix2_dict[line_part[0].lower()[0:2]] = counter_prefix2
                    counter_prefix2 = counter_prefix2+1
                if line_part[0].lower()[0:3] not in prefix3_dict.keys():
                    prefix3_dict[line_part[0].lower()[0:3]] = counter_prefix3
                    counter_prefix3 = counter_prefix3+1
            
                #suffix
                if line_part[0].lower()[-1] not in suffix1_dict.keys():
                    suffix1_dict[line_part[0].lower()[-1]] = counter_suffix1
                    counter_suffix1 = counter_suffix1+1
                if line_part[0].lower()[-2:] not in suffix2_dict.keys():
                    suffix2_dict[line_part[0].lower()[-2:]] = counter_suffix2
                    counter_suffix2 = counter_suffix2+1
                if line_part[0].lower()[-3:] not in suffix3_dict.keys():
                    suffix3_dict[line_part[0].lower()[-3:]] = counter_suffix3
                    counter_suffix3 = counter_suffix3+1
            
            elif len(line_part[0].lower())>=2:
                #prefix
                if line_part[0].lower()[0] not in prefix1_dict.keys():
                    prefix1_dict[line_part[0].lower()[0]] = counter_prefix1
                    counter_prefix1 = counter_prefix1+1
                if line_part[0].lower()[0:2] not in prefix2_dict.keys():
                    prefix2_dict[line_part[0].lower()[0:2]] = counter_prefix2
                    counter_prefix2 = counter_prefix2+1
                prefix3_dict["novalues"] = 9999
                
                #suffix
                if line_part[0].lower()[-1] not in suffix1_dict.keys():
                    suffix1_dict[line_part[0].lower()[-1]] = counter_suffix1
                    counter_suffix1 = counter_suffix1+1
                if line_part[0].lower()[-2:] not in suffix2_dict.keys():
                    suffix2_dict[line_part[0].lower()[-2:]] = counter_suffix2
                    counter_suffix2 = counter_suffix2+1
                suffix3_dict["novalues"] = 9999
            else:
                #prefix
                if line_part[0].lower()[0] not in prefix1_dict.keys():
                    prefix1_dict[line_part[0].lower()[0]] = counter_prefix1
                    counter_prefix1 = counter_prefix1+1
                prefix2_dict["novalues"] = 9999
                prefix3_dict["novalues"] = 9999
                #suffix
                if line_part[0].lower()[-1] not in suffix1_dict.keys():
                    suffix1_dict[line_part[0].lower()[-1]] = counter_suffix1
                    counter_suffix1 = counter_suffix1+1
                suffix2_dict["novalues"] = 9999
                suffix3_dict["novalues"] = 9999
        if line_part[1] not in postag_dict.keys():
            postag_dict[line_part[1]] = counter_postag
            counter_postag = counter_postag+1
        #print(t)
        sent.append(t)
    else:
        print("train sent = ")
        print(sent)
        train_sents.append(sent)
        sent = []
        counter = counter+1

train sent = 
[('Kegiatan', 'NN', 0), ('Rapat', 'NN', 0), ('Pimpinan', 'NN', 0), ('Nasional', 'JJ', 0), ('dibuka', 'VBT', 0), ('oleh', 'IN', 0), ('Ketua', 'NN', 0), ('Umum', 'JJ', 0), ('AMI', 'NN', 3), ('pusat', 'NN', 0), (',', ',', 0), ('Putu', 'NNP', 1), ('Supadma', 'NNP', 2), ('Rudana', 'NNP', 2), (',', ',', 2), ('MBA', 'NNP', 2), (',', ',', 0), ('Kamis', 'NN', 0), (',', ',', 0), ('3', 'CDP', 0), ('Oktober', 'NNP', 0), ('tahun', 'NN', 0), ('2019', 'CDP', 0), ('.', '.', 0)]
train sent = 
[('Menteri', 'NN', 0), ('Kelautan', 'NN', 0), ('dan', 'CC', 0), ('Perikanan', 'NN', 0), ('Susi', 'NNP', 1), ('Pudjiastuti', 'NNP', 2), ('meresmikan', 'VBT', 0), ('Cold', 'NN', 0), ('Storage', 'NN', 0), ('1.000', 'CDP', 0), ('ton', 'NN', 0), ('di', 'IN', 0), ('Penjaringan', 'NNP', 5), (',', ',', 0), ('Jakarta', 'NNP', 5), ('Utara', 'NN', 6), ('.', '.', 0)]
train sent = 
[('Sri', 'NN', 1), ('Mulyani', 'NNP', 2), ('Indrawati', 'NNP', 2), ('berkesempatan', 'VBI', 0), ('untuk', 'IN', 0), ('meresmikan', 'V

In [7]:
# data test
counter = 0
for line in test_lines:
    line = line.rstrip('\n')
    curr_tuple = ()
    if len(line)>1:
        line_part = line.split(" ")
        t = (line_part[0], line_part[1], convert_label(line_part[2]))
        #print(t)
        sent.append(t)
    else:
        print("test sent = ")
        print(sent)
        test_sents.append(sent)
        sent = []
        counter = counter+1

test sent = 
[('Wakil', 'NN', 0), ('Presiden', 'NN', 0), ('Jusuf', 'NN', 1), ('Kalla', 'NNP', 1), ('(', 'OP', 0), ('JK', 'NN', 0), (')', 'CP', 0), ('meresmikan', 'VBT', 0), ('Indonesian', 'NNP', 3), ('Agency', 'NN', 4), ('for', 'NN', 4), ('International', 'FW', 4), ('Development', 'NNP', 4), ('(', 'OP', 0), ('AID', 'NNP', 3), (')', 'CP', 0), ('atau', 'CC', 0), ('Lembaga', 'NN', 3), ('Kerja', 'NN', 4), ('Sama', 'JJ', 4), ('Pembangunan', 'NN', 4), ('Internasional', 'NNP', 4), ('Indonesia', 'NNP', 4), ('.', '.', 0), ('Menteri', 'NNP', 0), ('Energi', 'NNP', 0), ('dan', 'CC', 0), ('Sumber', 'NN', 0), ('Daya', 'NN', 0), ('Mineral', 'NNP', 0), ('(', 'OP', 0), ('ESDM', 'NNP', 0), (')', 'CP', 0), ('Arifin', 'NNP', 1), ('Tasrif', 'NNP', 2), ('meresmikan', 'VBT', 0), ('survei', 'NN', 0), ('seismik', 'NN', 0), ('untuk', 'IN', 0), ('mencari', 'VBT', 0), ('cadangan', 'NN', 0), ('minyak', 'NN', 0), ('dan', 'CC', 0), ('gas', 'NN', 0), ('(', 'OP', 0), ('migas)', 'NN', 0), ('.', '.', 0)]
test sent = 
[(

In [8]:
# kode untuk token/kata dan postag yang tidak muncul di data training, namun muncul di data testing
token_dict['unk'] = 9999
postag_dict['unk'] = 9999
prefix1_dict['novalues'] = 9999
prefix2_dict['novalues'] = 9999
prefix3_dict['novalues'] = 9999
suffix1_dict['novalues'] = 9999
suffix2_dict['novalues'] = 9999
suffix3_dict['novalues'] = 9999

In [9]:
# fungsi untuk ekstraksi fitur dari sebuah kalimat
def word2features(sent, i):  
    word = sent[i][0]
    postag = sent[i][1]
    if word.lower() not in token_dict.keys(): 
        word = 'unk'
    if postag not in postag_dict.keys():
        postag = 'unk'
    if len(word) >= 3:
        if word[0] not in prefix1_dict.keys():
            prefix1 = "novalues"
        else:
            prefix1 = word[0]
        if word[0:2] not in prefix2_dict.keys():
            prefix2 = "novalues"
        else:
            prefix2 = word[0:2]
        if word[0:3] not in prefix3_dict.keys():
            prefix3 = "novalues"
        else:
            prefix3 = word[0:3]
        if word[-1] not in suffix1_dict.keys():
            suffix1 = "novalues"
        else:
            suffix1 = word[-1]
        if word[-2:] not in suffix2_dict.keys():
            suffix2 = "novalues"
        else:
            suffix2 = word[-2:]
        if word[-3:] not in suffix3_dict.keys():
            suffix3 = "novalues"
        else:
            suffix3 = word[-3:]
    elif len(word) >=2:
        if word[0] not in prefix1_dict.keys():
            prefix1 = "novalues"
        else:
            prefix1 = word[0]
        if word[0:2] not in prefix2_dict.keys():
            prefix2 = "novalues"
        else:
            prefix2 = word[0:2]
        prefix3 = "novalues"
        if word[-1] not in suffix1_dict.keys():
            suffix1 = "novalues"
        else:
            suffix1 = word[-1]
        if word[-2:] not in suffix2_dict.keys():
            suffix2 = "novalues"
        else:
            suffix2 = word[-2:]
        suffix3 = "novalues"
    else:
        if word[0] not in prefix1_dict.keys():
            prefix1 = "novalues"
        else:
            prefix1 = word[0]
            prefix2 = "novalues"
            prefix3 = "novalues"
        if word[-1] not in suffix1_dict.keys():
            suffix1 = "novalues"
        else:
            suffix1 = word[-1]
            suffix2 = "novalues"
            suffix3 = "novalues"
    features = [
        token_dict[word.lower()], # fitur kata dalam bentuk lowercase
        word.isupper(), # fitur apakah karakter pertama token merupakan huruf kapital
        word.istitle(), # fitur apakah token merupakan title
        word.isdigit(), # fitur apakah token merupakan digit
        prefix1_dict[prefix1], #fitur 1 karakter pertama token
        prefix2_dict[prefix2], #fitur 2 karakter pertama token
        prefix3_dict[prefix3], #fitur 3 karakter pertama token
        suffix1_dict[suffix1], #fitur 1 karakter terakhir token
        suffix2_dict[suffix2], #fitur 2 karakter terakhir token
        suffix3_dict[suffix3], #fitur 3 karakter terakhir token
        postag_dict[postag] # fitur kode postag token
    ]
                
    return features

In [10]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]  

In [11]:
print('postag dictonary')
print(postag_dict)
print('token dictonary')
print(token_dict)

postag dictonary
{'NN': 0, 'JJ': 1, 'VBT': 2, 'IN': 3, ',': 4, 'NNP': 5, 'CDP': 6, '.': 7, 'CC': 8, 'VBI': 9, 'MD': 10, 'OP': 11, 'CP': 12, 'FW': 13, 'unk': 9999}
token dictonary
{'kegiatan': 0, 'rapat': 1, 'pimpinan': 2, 'nasional': 3, 'dibuka': 4, 'oleh': 5, 'ketua': 6, 'umum': 7, 'ami': 8, 'pusat': 9, ',': 10, 'putu': 11, 'supadma': 12, 'rudana': 13, 'mba': 14, 'kamis': 15, '3': 16, 'oktober': 17, 'tahun': 18, '2019': 19, '.': 20, 'menteri': 21, 'kelautan': 22, 'dan': 23, 'perikanan': 24, 'susi': 25, 'pudjiastuti': 26, 'meresmikan': 27, 'cold': 28, 'storage': 29, '1.000': 30, 'ton': 31, 'di': 32, 'penjaringan': 33, 'jakarta': 34, 'utara': 35, 'sri': 36, 'mulyani': 37, 'indrawati': 38, 'berkesempatan': 39, 'untuk': 40, 'museum': 41, 'perpustakaan': 42, 'baru': 43, 'milik': 44, 'bea': 45, 'cukai': 46, 'presiden': 47, 'jokowi': 48, 'akan': 49, 'jembatan': 50, 'holtekamp': 51, 'jayapura': 52, 'papua': 53, 'joko': 54, 'widodo': 55, 'pengoperasian': 56, 'palapa': 57, 'ring': 58, 'darmin':

In [12]:
# ekstraksi fitur data train
X_train = []
y_train = []
for s in train_sents:
    for i in range(len(s)):
        X_train.append(word2features(s,i))
        y_train.append(s[i][2])

In [13]:
# ekstraksi fitur data test
X_test = []
y_test = []
for s in test_sents:
    for i in range(len(s)):
        X_test.append(word2features(s,i))
        y_test.append(s[i][2])

In [14]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [15]:
# train classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
# Coba test, satu kata
print('xtest')
print(X_test[0])

xtest
[ 100    0    1    0 9999 9999 9999    2   56   81    0]


In [17]:
print(clf.predict(X_test[0].reshape(1,-1)))

[5]


In [18]:
# Coba test, keseluruhan data test
print('hasil klasifikasi data test:')
print(clf.predict(X_test))

hasil klasifikasi data test:
[5 5 5 5 0 3 0 0 5 5 0 5 5 0 3 0 0 5 5 5 5 5 5 0 5 0 0 0 0 0 0 0 0 0 0 0 0
 0 3 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 5 5 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 5 0]


In [19]:
print("akurasi : ", round(clf.score(X_test,y_test)*100,2), "%")

akurasi :  62.39 %


In [20]:
print(classification_report(y_test,clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.77      0.87      0.82        83
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         5
           3       0.33      0.14      0.20         7
           4       0.00      0.00      0.00        15
           5       0.00      0.00      0.00         1

    accuracy                           0.62       117
   macro avg       0.18      0.17      0.17       117
weighted avg       0.57      0.62      0.59       117



C:\Users\Asus\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
